In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Install required packages
!pip install -q wandb torch torchvision pandas numpy matplotlib seaborn scikit-learn mlflow pytorch_lightning pytorch_forecasting

# Set up Kaggle API
!pip install -q kaggle pytorch_forecasting pytorch_lightning dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82

In [7]:
# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
# Download the dataset
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -q walmart-recruiting-store-sales-forecasting.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 669MB/s]


In [9]:
!unzip -q train.csv.zip
!unzip -q stores.csv.zip
!unzip -q test.csv.zip
!unzip -q features.csv.zip

unzip:  cannot find or open stores.csv.zip, stores.csv.zip.zip or stores.csv.zip.ZIP.


In [22]:
# -*- coding: utf-8 -*-
"""
# Prophet Model Experiment Notebook

ეს ნოუთბუქი განკუთვნილია Walmart Store Sales Forecasting Kaggle-ის კონკურსისთვის Prophet მოდელის ექსპერიმენტებისთვის.
იგი მოიცავს მონაცემთა ჩატვირთვას, წინასწარ დამუშავებას, Prophet მოდელის ვარჯიშს, კროს-ვალიდაციას
და MLflow-ზე ექსპერიმენტების ლოგირებას.
"""
import mlflow
# --- 1. ბიბლიოთეკების იმპორტი ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error
import mlflow
import mlflow.pyfunc
import warnings
import dagshub
import joblib # დავამატე joblib ბიბლიოთეკა სერიალიზაციისთვის

warnings.filterwarnings("ignore")

print("საჭირო ბიბლიოთეკები წარმატებით იმპორტირებულია.")


საჭირო ბიბლიოთეკები წარმატებით იმპორტირებულია.


In [12]:

# --- 2. MLflow კონფიგურაცია ---
# MLflow Tracking URI-ის დაყენება. თუ იყენებთ ლოკალურ MLflow სერვერს, შეცვალეთ შესაბამისად.
# მაგალითად: "http://localhost:5000"
import dagshub
dagshub.init(repo_owner='ekvirika', repo_name='WalmartRecruiting', mlflow=True)
mlflow.autolog()


print(f"MLflow Tracking URI დაყენებულია: {mlflow.get_tracking_uri()}")

# ექსპერიმენტის სახელის დაყენება
experiment_name = "Prophet_Training"
mlflow.set_experiment(experiment_name)
print(f"MLflow ექსპერიმენტი დაყენებულია: {experiment_name}")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d96a3b54-4c07-4b5e-a688-57f7b864cfd2&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=b9a51f27d8eae2cc62f6856576ffbb7c85d32944f09a8ca568dd0665dcb044d1




Accessing as ekvirika

Initialized MLflow to track repo "ekvirika/WalmartRecruiting"

Repository ekvirika/WalmartRecruiting initialized!

2025/07/07 20:19:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/07 20:19:09 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2025/07/07 20:19:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2025/07/07 20:19:09 INFO mlflow.tracking.fluent: Experiment with name 'Prophet_Training' does not exist. Creating a new experiment.


MLflow Tracking URI დაყენებულია: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow
MLflow ექსპერიმენტი დაყენებულია: Prophet_Training


In [15]:

# --- 3. მონაცემთა ჩატვირთვა ---
# შენიშვნა: აქ უნდა მიუთითოთ თქვენი რეალური მონაცემთა ფაილების გზები.
# ამ მაგალითისთვის გამოყენებულია ფლეისჰოლდერები.
try:
    train_df = pd.read_csv("train.csv")
    test_df = pd.read_csv("test.csv")
    stores_df = pd.read_csv("stores.csv")
    features_df = pd.read_csv("features.csv")
except FileNotFoundError:
    print("შეცდომა: მონაცემთა ფაილები ვერ მოიძებნა. გთხოვთ, დარწმუნდით, რომ 'train.csv', 'test.csv', 'stores.csv', 'features.csv' არსებობს.")
    print("ამ ნოუთბუქის გასაშვებად საჭიროა ეს ფაილები ან ხელოვნური მონაცემების შექმნა.")
    exit() # გაჩერება თუ ფაილები ვერ მოიძებნა

# --- 4. მონაცემთა წინასწარი დამუშავება (MLflow Run: Prophet_Cleaning) ---
with mlflow.start_run(run_name="Prophet_Cleaning") as cleaning_run:
    print("მონაცემთა წინასწარი დამუშავება იწყება...")

    # თარიღების ფორმატის შეცვლა
    train_df['Date'] = pd.to_datetime(train_df['Date'])
    test_df['Date'] = pd.to_datetime(test_df['Date'])
    features_df['Date'] = pd.to_datetime(features_df['Date'])

    # Extract date-related features
    for df in [train_df, test_df]:
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Week'] = df['Date'].dt.isocalendar().week
        df['DayOfWeek'] = df['Date'].dt.dayofweek

    # Holiday flags for known holidays
    holiday_dates = {
        "Super Bowl": ["2010-02-12", "2011-02-11", "2012-02-10", "2013-02-08"],
        "Labor Day": ["2010-09-10", "2011-09-09", "2012-09-07", "2013-09-06"],
        "Thanksgiving": ["2010-11-26", "2011-11-25", "2012-11-23", "2013-11-29"],
        "Christmas": ["2010-12-31", "2011-12-30", "2012-12-28", "2013-12-27"]
    }


    for holiday, dates in holiday_dates.items():
        for df in [train_df, test_df]:
            df[holiday] = df['Date'].isin(pd.to_datetime(dates)).astype(int)


    # მონაცემთა გაერთიანება
    # გაერთიანება features_df-თან
    train_df = pd.merge(train_df, features_df, on=['Store', 'Date', 'IsHoliday'], how='left', suffixes=('_train', '_features'))
    test_df = pd.merge(test_df, features_df, on=['Store', 'Date', 'IsHoliday'], how='left', suffixes=('_test', '_features'))

    # გაერთიანება stores_df-თან
    train_df = pd.merge(train_df, stores_df, on='Store', how='left')
    test_df = pd.merge(test_df, stores_df, on='Store', how='left')

    # Weekly_Sales-ის უარყოფითი მნიშვნელობების დამუშავება (თუ არსებობს)
    # Prophet არ უჭერს მხარს უარყოფით მნიშვნელობებს.
    train_df['Weekly_Sales'] = train_df['Weekly_Sales'].apply(lambda x: max(0, x))

    # დაკარგული მნიშვნელობების დამუშავება
    # MarkDown-ის სვეტები ხშირად შეიცავს NaN-ებს, რადგან აქციები არ ყოფილა.
    # შევავსოთ 0-ებით ან სხვა შესაბამისი მნიშვნელობით.
    markdown_cols = [col for col in train_df.columns if 'MarkDown' in col]
    for col in markdown_cols:
        train_df[col] = train_df[col].fillna(0)
        test_df[col] = test_df[col].fillna(0)

    # CPI, Unemployment, Temperature, Fuel_Price-ის დაკარგული მნიშვნელობების შევსება
    # შევავსოთ წინა მნიშვნელობით ან საშუალოთი, რადგან ეს დროითი სერიების მონაცემებია.
    for col in ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']:
        train_df[col] = train_df[col].fillna(method='ffill').fillna(method='bfill')
        test_df[col] = test_df[col].fillna(method='ffill').fillna(method='bfill')
        # თუ მაინც დარჩა NaN (მაგ. სერიის დასაწყისში), შევავსოთ საშუალოთი
        train_df[col] = train_df[col].fillna(train_df[col].mean())
        test_df[col] = test_df[col].fillna(test_df[col].mean())

    # MLflow-ში ლოგირება
    mlflow.log_param("initial_train_rows", train_df.shape[0])
    mlflow.log_param("initial_test_rows", test_df.shape[0])
    mlflow.log_param("regressors_used", ', '.join(markdown_cols + ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']))

    print("მონაცემთა წინასწარი დამუშავება დასრულდა.")
    print(f"დამუშავებული სატრენინგო მონაცემების ნიმუში:\n{train_df.head()}")
    print(f"დამუშავებული სატესტო მონაცემების ნიმუში:\n{test_df.head()}")


მონაცემთა წინასწარი დამუშავება იწყება...
მონაცემთა წინასწარი დამუშავება დასრულდა.
დამუშავებული სატრენინგო მონაცემების ნიმუში:
   Store  Dept       Date  Weekly_Sales  IsHoliday  Year  Month  Week  \
0      1     1 2010-02-05      24924.50      False  2010      2     5   
1      1     1 2010-02-12      46039.49       True  2010      2     6   
2      1     1 2010-02-19      41595.55      False  2010      2     7   
3      1     1 2010-02-26      19403.54      False  2010      2     8   
4      1     1 2010-03-05      21827.90      False  2010      3     9   

   DayOfWeek  Super Bowl  ...  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  \
0          4           0  ...       2.572        0.0        0.0        0.0   
1          4           1  ...       2.548        0.0        0.0        0.0   
2          4           0  ...       2.514        0.0        0.0        0.0   
3          4           0  ...       2.561        0.0        0.0        0.0   
4          4           0  ...       2.625    

In [26]:
# --- 5. მოდელის ვარჯიში და კროს-ვალიდაცია ყველა Store/Dept კომბინაციისთვის ---
# მიიღეთ ყველა უნიკალური Store-Dept კომბინაცია
unique_store_dept_combinations = train_df[['Store', 'Dept']].drop_duplicates().values

print(f"სულ უნიკალური Store-Dept კომბინაციები: {len(unique_store_dept_combinations)}")

# დამატებითი რეგრესორების მომზადება
regressor_cols = ['IsHoliday', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] + markdown_cols

for store_id, dept_id in unique_store_dept_combinations:
    # შექმენით მშობელი გაშვება თითოეული Store/Dept კომბინაციისთვის
    with mlflow.start_run(run_name=f"Prophet_Store_{store_id}_Dept_{dept_id}_Experiment") as parent_run:
        print(f"\n--- დაწყებულია ექსპერიმენტი Store {store_id}, Dept {dept_id}-ისთვის ---")

        # მონაცემების ფილტრაცია მიმდინარე Store/Dept-ისთვის
        train_filtered = train_df[(train_df['Store'] == store_id) & (train_df['Dept'] == dept_id)].copy()
        test_filtered = test_df[(test_df['Store'] == store_id) & (test_df['Dept'] == dept_id)].copy()

        # Prophet-ისთვის საჭირო სვეტების გადარქმევა: 'ds' (თარიღი) და 'y' (სამიზნე)
        train_prophet_full = train_filtered[['Date', 'Weekly_Sales'] + regressor_cols].rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'})
        test_prophet_full = test_filtered[['Date'] + regressor_cols].rename(columns={'Date': 'ds'})

        # დარწმუნდით, რომ IsHoliday არის რიცხვითი
        train_prophet_full['IsHoliday'] = train_prophet_full['IsHoliday'].astype(int)
        test_prophet_full['IsHoliday'] = test_prophet_full['IsHoliday'].astype(int)

        # შეამოწმეთ, არის თუ არა საკმარისი მონაცემები
        if len(train_prophet_full) < 2: # Prophet-ს მინიმუმ 2 წერტილი სჭირდება
            print(f"გამოტოვება: არასაკმარისი მონაცემები Store {store_id}, Dept {dept_id}-ისთვის. (სულ {len(train_prophet_full)} ჩანაწერი)")
            mlflow.log_param("status", "skipped_insufficient_data")
            mlflow.log_param("num_records", len(train_prophet_full))
            continue # გადადით შემდეგ კომბინაციაზე

        # --- 5.1. მოდელის ვარჯიში (Nested MLflow Run: Prophet_StoreX_DeptY_Training) ---
        with mlflow.start_run(run_name=f"Prophet_Store_{store_id}_Dept_{dept_id}_Training", nested=True) as training_run:
            print(f"Prophet მოდელის ვარჯიში იწყება Store {store_id}, Dept {dept_id}-ისთვის...")

            # Prophet მოდელის ინიციალიზაცია
            model_params = {
                'seasonality_mode': 'multiplicative',
                'changepoint_prior_scale': 0.05,
                'weekly_seasonality': True,
                'daily_seasonality': False,
                'yearly_seasonality': True
            }

            m = Prophet(**model_params)

            # დამატებითი რეგრესორების დამატება
            for col in regressor_cols:
                if col in train_prophet_full.columns and col in test_prophet_full.columns:
                    m.add_regressor(col)
                    mlflow.log_param(f"regressor_{col}", "added")
                else:
                    print(f"გაფრთხილება: რეგრესორი '{col}' არ მოიძებნა ყველა მონაცემთა ნაკრებში Store {store_id}, Dept {dept_id}-ისთვის.")

            # მოდელის ვარჯიში
            m.fit(train_prophet_full)

            print(f"Prophet მოდელის ვარჯიში დასრულდა Store {store_id}, Dept {dept_id}-ისთვის.")

            # MLflow-ში პარამეტრების და ლოგირება
            for param, value in model_params.items():
                mlflow.log_param(param, value)
            mlflow.log_param("filtered_store_id", store_id)
            mlflow.log_param("filtered_dept_id", dept_id)
            mlflow.log_param("final_train_rows_prophet", train_prophet_full.shape[0])
            mlflow.log_param("final_test_rows_prophet", test_prophet_full.shape[0])

            # მოდელის შენახვა MLflow-ში
            mlflow.pyfunc.log_model(
                artifact_path="prophet_model",
                python_model=mlflow.pyfunc.PythonModel(),
                artifacts={"prophet_model": m},
                # code_path=[__file__],
                conda_env={
                    "channels": ["defaults", "conda-forge"],
                    "dependencies": [
                        "python=3.9", # შეცვალეთ თქვენი python ვერსიით
                        "pandas",
                        "numpy",
                        "prophet",
                        "scikit-learn",
                        "matplotlib",
                        "plotly",
                        "mlflow"
                    ]
                }
            )
            print(f"Prophet მოდელი შენახულია MLflow-ში Store {store_id}, Dept {dept_id}-ისთვის.")

            # პროგნოზირება სატრენინგო მონაცემებზე (ვიზუალიზაციისთვის)
            forecast_train = m.predict(train_prophet_full[['ds'] + [col for col in regressor_cols if col in train_prophet_full.columns]])

            # ვიზუალიზაცია
            fig = m.plot(forecast_train)
            plt.title(f"Prophet პროგნოზი (Store {store_id}, Dept {dept_id})")
            plt.xlabel("თარიღი")
            plt.ylabel("კვირის გაყიდვები")
            mlflow.log_figure(fig, "prophet_forecast_train.png")
            plt.close(fig)

            # კომპონენტების ვიზუალიზაცია
            fig2 = m.plot_components(forecast_train)
            mlflow.log_figure(fig2, "prophet_components.png")
            plt.close(fig2)

            # --- 5.2. კროს-ვალიდაცია (Nested MLflow Run: Prophet_StoreX_DeptY_Cross_Validation) ---
            with mlflow.start_run(run_name=f"Prophet_Store_{store_id}_Dept_{dept_id}_Cross_Validation", nested=True) as cv_run:
                print(f"Prophet კროს-ვალიდაცია იწყება Store {store_id}, Dept {dept_id}-ისთვის...")

                # კროს-ვალიდაციის პარამეტრები
                # initial: საწყისი სატრენინგო პერიოდი
                # period: ყოველი შემდგომი გაფართოების პერიოდი
                # horizon: პროგნოზის ჰორიზონტი
                # დარწმუნდით, რომ initial საკმარისად დიდია.
                # Prophet-ის დოკუმენტაცია გვირჩევს მინიმუმ 20-ჯერ ჰორიზონტის ზომა.
                # თუ მონაცემები მცირეა, ეს შეიძლება გამოიწვიოს შეცდომა.
                # ვცადოთ დინამიური initial
                min_initial_weeks = 20 # Prophet-ის რეკომენდაცია
                initial_weeks = max(min_initial_weeks, int(len(train_prophet_full) * 0.7)) # მინიმუმ 70% ან 20 კვირა
                cv_initial = f"{initial_weeks} W"
                cv_period = "4 W"
                cv_horizon = "12 W"

                print(f"კროს-ვალიდაციის პარამეტრები: initial={cv_initial}, period={cv_period}, horizon={cv_horizon}")

                # MLflow-ში ლოგირება
                mlflow.log_param("cv_initial", cv_initial)
                mlflow.log_param("cv_period", cv_period)
                mlflow.log_param("cv_horizon", cv_horizon)

                # კროს-ვალიდაციის გაშვება
                try:
                    df_cv = cross_validation(
                        m,
                        initial=cv_initial,
                        period=cv_period,
                        horizon=cv_horizon,
                        cutoffs=None,
                        parallel="processes"
                    )
                    print("კროს-ვალიდაცია დასრულდა.")
                    print(f"კროს-ვალიდაციის შედეგების ნიმუში:\n{df_cv.head()}")

                    # შესრულების მეტრიკების გამოთვლა
                    df_p = performance_metrics(df_cv)
                    print(f"შესრულების მეტრიკების ნიმუში:\n{df_p.head()}")

                    # შეფასების მეტრიკების ლოგირება MLflow-ში
                    mlflow.log_metric("mae_cv_mean", df_p['mae'].mean())
                    mlflow.log_metric("rmse_cv_mean", df_p['rmse'].mean())
                    mlflow.log_metric("mape_cv_mean", df_p['mape'].mean())
                    mlflow.log_metric("smape_cv_mean", df_p['smape'].mean())

                    # ვიზუალიზაცია: შეცდომის მეტრიკები დროის მიხედვით
                    fig_metrics = make_subplots(rows=2, cols=1, shared_xaxes=True,
                                                subplot_titles=("MAE დროის მიხედვით", "RMSE დროის მიხედვით"))
                    fig_metrics.add_trace(go.Scatter(x=df_p['horizon'], y=df_p['mae'], mode='lines', name='MAE'), row=1, col=1)
                    fig_metrics.add_trace(go.Scatter(x=df_p['horizon'], y=df_p['rmse'], mode='lines', name='RMSE'), row=2, col=1)
                    fig_metrics.update_layout(title_text=f"Prophet კროს-ვალიდაციის მეტრიკები (Store {store_id}, Dept {dept_id})", height=600)
                    mlflow.log_figure(fig_metrics, "prophet_cv_metrics.png")

                except ValueError as e:
                    print(f"შეცდომა კროს-ვალიდაციისას Store {store_id}, Dept {dept_id}-ისთვის: {e}")
                    print("ეს შეიძლება მოხდეს, თუ მონაცემთა ნაკრები ძალიან მცირეა კროს-ვალიდაციისთვის მითითებული პარამეტრებით.")
                    mlflow.log_param("cv_error", str(e))
                except Exception as e:
                    print(f"მოულოდნელი შეცდომა კროს-ვალიდაციისას Store {store_id}, Dept {dept_id}-ისთვის: {e}")
                    mlflow.log_param("cv_error", str(e))

            # --- 5.3. მოდელის რეგისტრაცია (MLflow Model Registry) ---
            # რეგისტრაციისთვის საჭიროა run_id, საიდანაც მოდელი დაილოგა.
            try:
                registered_model = mlflow.register_model(
                    model_uri=f"runs:/{training_run.info.run_id}/prophet_model",
                    name=f"WalmartSalesProphetModel_Store_{store_id}_Dept_{dept_id}"
                )
                print(f"მოდელი 'WalmartSalesProphetModel_Store_{store_id}_Dept_{dept_id}' წარმატებით დარეგისტრირდა Model Registry-ში.")
                print(f"ვერსია: {registered_model.version}")
            except Exception as e:
                print(f"შეცდომა მოდელის რეგისტრაციისას Store {store_id}, Dept {dept_id}-ისთვის: {e}")
                print("დარწმუნდით, რომ MLflow Tracking Server გაშვებულია და გაქვთ წვდომა Model Registry-ზე.")

print("\nყველა Prophet მოდელის ექსპერიმენტი დასრულდა.")


სულ უნიკალური Store-Dept კომბინაციები: 3331

--- დაწყებულია ექსპერიმენტი Store 1, Dept 1-ისთვის ---
Prophet მოდელის ვარჯიში იწყება Store 1, Dept 1-ისთვის...


DEBUG:cmdstanpy:input tempfile: /tmp/tmphqh0gn0f/5tdcfd8l.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphqh0gn0f/75gtmayj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=82078', 'data', 'file=/tmp/tmphqh0gn0f/5tdcfd8l.json', 'init=/tmp/tmphqh0gn0f/75gtmayj.json', 'output', 'file=/tmp/tmphqh0gn0f/prophet_modelo8couty8/prophet_model-20250707203545.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
20:35:45 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
20:35:45 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prophet მოდელის ვარჯიში დასრულდა Store 1, Dept 1-ისთვის.


2025/07/07 20:35:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Prophet_Store_1_Dept_1_Training at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0/runs/69b68899f52d459a9f8b89756fb7b45b
🧪 View experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0
🏃 View run Prophet_Store_1_Dept_1_Experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0/runs/70e872430f584da3bd3df1c8d2602381
🧪 View experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}

In [ ]:

# --- 6. კროს-ვალიდაცია (MLflow Run: Prophet_Cross_Validation) ---
with mlflow.start_run(run_name="Prophet_Cross_Validation", nested=True) as cv_run:
    print("Prophet კროს-ვალიდაცია იწყება...")

    # კროს-ვალიდაციის პარამეტრები
    # initial: საწყისი სატრენინგო პერიოდი
    # period: ყოველი შემდგომი გაფართოების პერიოდი
    # horizon: პროგნოზის ჰორიზონტი
    cv_initial = f"{int(len(train_prophet_full) * 0.7)} W" # 70% საწყისი ტრენინგი
    cv_period = "4 W" # ყოველ 4 კვირაში
    cv_horizon = "12 W" # 12 კვირიანი პროგნოზი

    print(f"კროს-ვალიდაციის პარამეტრები: initial={cv_initial}, period={cv_period}, horizon={cv_horizon}")

    # MLflow-ში ლოგირება
    mlflow.log_param("cv_initial", cv_initial)
    mlflow.log_param("cv_period", cv_period)
    mlflow.log_param("cv_horizon", cv_horizon)

    # კროს-ვალიდაციის გაშვება
    try:
        df_cv = cross_validation(
            m,
            initial=cv_initial,
            period=cv_period,
            horizon=cv_horizon,
            cutoffs=None, # ავტომატური cutoffs
            parallel="processes" # პარალელური შესრულება
        )
        print("კროს-ვალიდაცია დასრულდა.")
        print(f"კროს-ვალიდაციის შედეგების ნიმუში:\n{df_cv.head()}")

        # შესრულების მეტრიკების გამოთვლა
        df_p = performance_metrics(df_cv)
        print(f"შესრულების მეტრიკების ნიმუში:\n{df_p.head()}")

        # შეფასების მეტრიკების ლოგირება MLflow-ში
        mlflow.log_metric("mae_cv_mean", df_p['mae'].mean())
        mlflow.log_metric("rmse_cv_mean", df_p['rmse'].mean())
        mlflow.log_metric("mape_cv_mean", df_p['mape'].mean())
        mlflow.log_metric("smape_cv_mean", df_p['smape'].mean())

        # ვიზუალიზაცია: შეცდომის მეტრიკები დროის მიხედვით
        fig_metrics = make_subplots(rows=2, cols=1, shared_xaxes=True,
                                    subplot_titles=("MAE დროის მიხედვით", "RMSE დროის მიხედვით"))
        fig_metrics.add_trace(go.Scatter(x=df_p['horizon'], y=df_p['mae'], mode='lines', name='MAE'), row=1, col=1)
        fig_metrics.add_trace(go.Scatter(x=df_p['horizon'], y=df_p['rmse'], mode='lines', name='RMSE'), row=2, col=1)
        fig_metrics.update_layout(title_text="Prophet კროს-ვალიდაციის მეტრიკები", height=600)
        mlflow.log_figure(fig_metrics, "prophet_cv_metrics.png")

    except ValueError as e:
        print(f"შეცდომა კროს-ვალიდაციისას: {e}")
        print("ეს შეიძლება მოხდეს, თუ მონაცემთა ნაკრები ძალიან მცირეა კროს-ვალიდაციისთვის მითითებული პარამეტრებით.")
        mlflow.log_param("cv_error", str(e))
    except Exception as e:
        print(f"მოულოდნელი შეცდომა კროს-ვალიდაციისას: {e}")
        mlflow.log_param("cv_error", str(e))

print("Prophet მოდელის ექსპერიმენტი დასრულდა.")


In [ ]:

# --- 7. მოდელის რეგისტრაცია (MLflow Model Registry) ---
# ეს ნაბიჯი უნდა შესრულდეს მხოლოდ საუკეთესო მოდელისთვის.
# ამ ნოუთბუქში, ჩვენ ვრეგისტრირებთ Prophet მოდელს.
# model_inference.ipynb გამოიყენებს ამ რეგისტრირებულ მოდელს.

# დარწმუნდით, რომ training_run აქტიურია ან გამოიყენეთ run_id training_run-დან
# თუ გსურთ მოდელის რეგისტრაცია ცალკე run-ში, შექმენით ახალი run.
# ამ მაგალითისთვის, ვრეგისტრირებთ მოდელს, რომელიც ახლახან დავატრენინგეთ.

# MLflow-ში შენახული მოდელის ჩატვირთვა MLflow-ის Artifacts-დან
# რადგან Prophet არ არის პირდაპირ PyTorch/Sklearn, ჩვენ ის შევინახეთ როგორც pyfunc
# და ახლა უნდა ჩავტვირთოთ შესაბამისად.
# ეს ნაწილი შეიძლება იყოს model_inference.ipynb-შიც.

# მოდელის რეგისტრაცია
# რეგისტრაციისთვის საჭიროა run_id, საიდანაც მოდელი დაილოგა.
# training_run.info.run_id არის run_id training_run-ისთვის.
try:
    registered_model = mlflow.register_model(
        model_uri=f"runs:/{training_run.info.run_id}/prophet_model",
        name="WalmartSalesProphetModel"
    )
    print(f"მოდელი 'WalmartSalesProphetModel' წარმატებით დარეგისტრირდა Model Registry-ში.")
    print(f"ვერსია: {registered_model.version}")
except Exception as e:
    print(f"შეცდომა მოდელის რეგისტრაციისას: {e}")
    print("დარწმუნდით, რომ MLflow Tracking Server გაშვებულია და გაქვთ წვდომა Model Registry-ზე.")